# Running the Notebook Server

The IPython notebook server is a custom web server that runs the notebook web application. Most of the time, users run the notebook server on their local computer using IPython's command line interface.

## Starting the notebook server using the command line

You can start the notebook server from the command line (Terminal on Mac/Linux, CMD prompt on Windows) by running the following command: 

    ipython notebook

This will print some information about the notebook server in your terminal, including the URL of the web application (by default, `http://127.0.0.1:8888`). It will then open your default web browser to this URL.

When the notebook opens, you will see the **notebook dashboard**, which will show a list of the notebooks and subdirectories in the directory where the notebook server was started. As of IPython 2.0, the dashboard allows you to navigate to different subdirectories. Because of this, it is no longer necessary to start a separate notebook server for each subdirectory. Most of the time, you will want to start a notebook server in the highest directory in your filesystem where notebooks can be found. Often this will be your home directory.

You can start more than one notebook server at the same time. By default, the first notebook server starts on port 8888 and later notebook servers search for open ports near that one.

You can also specify the port manually:

    ipython notebook --port 9999

Or start notebook server without opening a web browser.

    ipython notebook --no-browser

The notebook server has a number of other command line arguments that can be displayed with the `--help` flag: 

    ipython notebook --help

<div class="alert alert-failure">
It used to be possible to specify kernel options, such as <code>--pylab inline</code> from the command line. This is deprecated in IPython 2.0 and will be removed in IPython 3.0. To enable matplotlib based plotting for the Python kernel use the <code>%matplotlib</code> magic command.
</div>



## Configuring the IPython Notebook

The notebook web server can also be configured using IPython profiles and configuration files. The Notebook web server configuration options are set in a file named `ipython_notebook_config.py` in your IPython *profile directory*. The profile directory is a subfolder of your IPython directory, which itself is usually `.ipython` in your home directory.

You can display the location of your default profile directory by running the command:

In [7]:
!ipython profile locate default

/Users/bgranger/.ipython/profile_default


The default version of `ipython_notebook_config.py` lists all of the options available along with documentation for each. Changes made to that file will affect all notebook servers run under that profile. Command line options always override those set in configuration files.

You can create a new profile:

In [3]:
!ipython profile create my_profile

[ProfileCreate] Generating default config file: u'/Users/bgranger/.ipython/profile_my_profile/ipython_config.py'
[ProfileCreate] Generating default config file: u'/Users/bgranger/.ipython/profile_my_profile/ipython_qtconsole_config.py'
[ProfileCreate] Generating default config file: u'/Users/bgranger/.ipython/profile_my_profile/ipython_notebook_config.py'
[ProfileCreate] Generating default config file: u'/Users/bgranger/.ipython/profile_my_profile/ipython_nbconvert_config.py'


And then view its location:

In [5]:
!ipython profile locate my_profile

/Users/bgranger/.ipython/profile_my_profile


To start the notebook server using a given profile, run the following:

    ipython notebook --profile=my_profile

More details about IPython configuration files and profiles can be found [here](http://ipython.org/ipython-doc/dev/config/intro.html).

## Securing the notebook server

The IPython Notebook allows arbitrary code execution on the computer running it. Thus, the notebook web server should never be run on the open internet without first securing it. By default, the notebook server only listens on local network interface (`127.0.0.1`) There are two steps required to secure the notebook server:

1. Setting a password
2. Encrypt network traffic using SSL

### Setting a password

You can protect your notebook server with a simple single password by setting the `NotebookApp.password` configurable. You can prepare a hashed password using the function `IPython.lib.passwd`:

In [1]:
from IPython.lib import passwd
password = passwd("secret")
password

'sha1:6c2164fc2b22:ed55ecf07fc0f985ab46561483c0e888e8964ae6'

You can then add this to your `ipython_notebook_config.py`:

```python
# Password to use for web authentication
c = get_config()
c.NotebookApp.password = 
u'sha1:6c2164fc2b22:ed55ecf07fc0f985ab46561483c0e888e8964ae6'
```

### Using SSL/HTTPS

When using a password, it is a good idea to also use SSL, so that your 
password is not sent unencrypted by your browser to the web server. When running the notebook on the public internet this is absolutely required.

The first step is to generate an SSL certificate. A self-signed certificate can be generated with ``openssl``.  For example, the following command will create a certificate valid for 365 days with both the key and certificate data written to the same file:

    openssl req -x509 -nodes -days 365 -newkey rsa:1024 -keyout mycert.pem -out mycert.pem

In most cases, you should run this command in your profile directory, which will make it easy to use the generated key and certificate.

When you connect to a notebook server over HTTPS using a self-signed certificate, your browser will warn you of a dangerous certificate because it is self-signed.  If you want to have a fully compliant certificate that will not raise warnings, it is possible (but rather involved) to obtain one,
as explained in detail in [this tutorial](http://arstechnica.com/security/news/2009/12/how-to-get-set-with-a-secure-sertificate-for-free.ars)
	
When you enable SSL support, you will need to access the notebook server over ``https://``, rather than plain ``http://``.  The startup message from the notebook server prints the correct URL, but it is easy to overlook and think the server is for some reason non-responsive.

Once you have generated the key and certificate, you can configure the notebook server to use them, by adding the following to `ipython_notebook_config.py`:

```python
# The full path to an SSL/TLS certificate file.
c.NotebookApp.certfile = u'/Users/bgranger/.ipython/profile_my_profile/mycert.crt'

# The full path to a private key file for usage with SSL/TLS.
c.NotebookApp.keyfile = u'/Users/bgranger/.ipython/profile_my_profile/mycert.key'
```

## Running a public notebook server

<div class="alert alert-error">
Don't run a public notebook server unless you first secure it with a password and SSL/HTTPS as described above
</div>

By default the notebook server only listens on the `localhost/127.0.0.1` network interface. If you want to connect to the notebook from another computers, or over the internet, you need to configure the notebook server to listen on all network interfaces and not open the browser. You will often also want to disable the automatic launching of the web browser.

This can be accomplished by passing a command line options.

    ipython notebook --ip=* --no-browser

You can also add the following to your`ipython_notebook_config.py` file:

```python
c.NotebookApp.ip = '*'
c.NotebookApp.open_browser = False
```

## Running with a different URL prefix

The notebook dashboard typically lives at the URL `http://localhost:8888/tree`. If you prefer that it lives, together with the 
rest of the notebook web application, under a base URL prefix, such as `http://localhost:8888/ipython/tree`, you can do so by adding the following lines to your `ipython_notebook_config.py` file.

```python
c.NotebookApp.base_url = '/ipython/'
c.NotebookApp.webapp_settings = {'static_url_prefix':'/ipython/static/'}
```

## Using a different notebook store

By default, the notebook server stores the notebook documents that it saves as files in the working directory of the notebook server, also known as the
`notebook_dir`. This  logic is implemented in the `FileNotebookManager` class. However, the server can be configured to use a different notebook manager class, which can store the notebooks in a different format. 

The [bookstore](https://github.com/rgbkrk/bookstore) package currently allows users to store notebooks on Rackspace CloudFiles or OpenStack Swift based object stores.

Writing a notebook manager is as simple as extending the base class `NotebookManager`. The [simple_notebook_manager](https://github.com/khinsen/simple_notebook_manager) provides a great example
of an in memory notebook manager, created solely for the purpose of
illustrating the notebook manager API.

## Known issues

When behind a proxy, especially if your system or browser is set to autodetect the proxy, the notebook web application might fail to connect to the server's websockets, and present you with a warning at startup. In this case, you need to configure your system not to use the proxy for the server's address.

For example, in Firefox, go to the Preferences panel, Advanced section,
Network tab, click 'Settings...', and add the address of the notebook server
to the 'No proxy for' field.